## check data

In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("artifacts/data_ingestion/iris.csv")
df.head()

In [7]:
X=df.drop("name", axis=1)
y=df["name"]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
list(df.columns)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#no null value
df.isnull().value_counts()

In [15]:
df.isnull().sum()

df.columns

def checknull(data):
    for i in range(len(data.columns)):
        if(int(df[df.columns[i]].isnull().sum())==0):
            return True


In [ ]:
df.iloc[:,:-1]

In [ ]:
df.iloc[:,-1]

In [ ]:
if(int(df[df.columns[0]].isnull().sum()) ==0 ):
    print("hhhhh")



### EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from src.irisdataprediction.utils.common import save_figure
%matplotlib inline

# to set default font size

# plt.rc('font', size=14)
# plt.rc('axes', labelsize=14, titlesize=14)
# plt.rc('legend', fontsize=14)
# plt.rc('xtick',labelsize=10)
# plt.rc('ytick',labelsize=10)
df.hist(bins=50,figsize=(12,8))
# save_figure("test", tight_layout=True, fig_extension="png", resolution=300,image_path="")
# print("---------------",type(plt).__name__)


In [ ]:
type(df[df.columns[3]].iloc[3])

In [ ]:
for i in range(len(df.columns)):
    if type(df[df.columns[i]].iloc[i]) !=str:
        print(df.columns[i])
        

In [ ]:
df.info()

In [ ]:
df.groupby('name')['name'].count()

In [ ]:
df['name'].describe()

In [18]:
from pathlib import Path
IMAGE_PATH=Path() /"images"/"EDA"

In [19]:
#separate dataset into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test=train_test_split(X,y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train.corr()

In [ ]:
import seaborn as sns

plt.figure(figsize=(5,3))
cor=df.iloc[:,:-1].corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(5,3))
cor=X_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 8))
i=0
for ax, column in zip(axes.flat, df.columns):

    if type(df[column].iloc[i]) !=str:
        sns.boxplot(df[column],ax=ax)
        ax.set_title(column)
        i +=1
fig.tight_layout()
plt.show()

In [23]:
# with the following function we can select highly correlated features
# It will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr=set() #set of the names of correlated columns
    corr_matrix=dataset.corr()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])> threshold: #absolute is to compare +ve and -ve correlated columns both with threshold.
                colname= corr_matrix.columns[i] #getting the name of the column
                col_corr.add(colname)
    
    return col_corr

In [ ]:
#will use 85% as threshold
corr_features=correlation(X_train, 0.85)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
#dropping the correlated features

X_train.drop(corr_features, axis=1)
X_test.drop(corr_features,axis=1)

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict  #this is to hold data from schema.yaml
    image_path: Path

In [28]:
import os
import sys
import pandas as pd
from src.irisdataprediction import logger
from src.irisdataprediction.constants import *
from src.irisdataprediction.utils.common import read_yaml, create_directories,save_figure
from src.irisdataprediction.exception import IrisPredictionException
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema= self.schema.COLUMNS

        create_directories([config.root_dir])
        create_directories([config.image_path])

        data_validation_config=DataValidationConfig(
            root_dir= config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
            image_path=config.image_path
        )

        return data_validation_config

In [77]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config= config
        self.dataset=pd.read_csv(self.config.unzip_data_dir)

    def validate_all_columns(self)->bool:
        try:
            validation_status= None
            dataset=self.dataset

            # data= pd.read_csv(self.config.unzip_data_dir)
            all_cols= list(dataset.columns)
            all_schema=self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status= True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status

        except Exception as e:
            raise IrisPredictionException(e, sys)

    def eda(self):

        dataset=self.dataset
        # to set default font size        
        dataset.hist(bins=50,figsize=(12,8))
        save_figure("attribute_histogram_plots",self.config.image_path,tight_layout=True, fig_extension="png", resolution=300)
        plt.show()

        # check dataset embalance
        dataset['name'].value_counts().plot(kind='bar')
        plt.title("class distribution")
        plt.xlabel("class")
        plt.ylabel('Count')
        save_figure("class_count_check_data_imbalance",self.config.image_path,tight_layout=True, fig_extension="png", resolution=300)
        plt.show()

        #check of outliers
        #set the figure size
        fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 8))
        i=0
        for ax, column in zip(axes.flat, dataset.columns):

            if type(dataset[column].iloc[i]) !=str:
                sns.boxplot(dataset[column],ax=ax)
                ax.set_title(column)
            i +=1
        fig.tight_layout()
        save_figure("check_outliers",self.config.image_path,tight_layout=True, fig_extension="png", resolution=300)
        plt.show()
        

    def validate_null_value(self):
        try:
            validation_status= None
            dataset=self.dataset

            for i in range(len(dataset.columns)):
                if(int(df[df.columns[i]].isnull().sum())==0):
                    validation_status= True

                else:
                    validation_status= False
                    
                break
            if(validation_status):
                    with open (self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")

        except Exception as e:
            raise IrisPredictionException(e, sys)
            

In [ ]:
try:
    config= ConfigurationManager()
    data_validation_config= config.get_data_validation_config()
    data_validation= DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.eda()
    data_validation.validate_null_value()
except Exception as e:
    raise IrisPredictionException(e, sys)
